## Transfer Learning

In [3]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions

### 1. Load all images

In [17]:
import os
import numpy as np

X = []
y = []
path = "train"
for subfolder in os.listdir(path):
    for filename in os.listdir(path + os.sep + subfolder):
        y.append(subfolder)
        fn = path + os.sep + subfolder + os.sep + filename
        im = image.load_img(fn, target_size=(224, 224))
        a = image.img_to_array(im)
        X.append(preprocess_input(a))

X = np.array(X)
y = np.array(y)

In [18]:
X.shape, y.shape

((85, 224, 224, 3), (85,))

### 2. Create Mobile Net without top

In [22]:
m = MobileNetV2(include_top=False, input_shape=(224, 224, 3))

### 3. Predict

In [24]:
Xcnn = m.predict(X)
Xcnn = Xcnn.reshape(85, 7 * 7 * 1280)
Xcnn.shape

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step


(85, 62720)

### 4. train RandomForest

In [27]:
#!uv add scikit-learn

In [35]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, max_depth=3)
rf.fit(Xcnn, y)
rf.score(Xcnn, y)

0.9764705882352941

### 5. Cross-validate

In [36]:
from sklearn.model_selection import cross_val_score

cross_val_score(rf, Xcnn, y, cv=5)

C:\Users\team-\Desktop\ml_code\.venv\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


array([0.88235294, 0.88235294, 0.82352941, 0.76470588, 0.82352941])

### 6. Regularize

### 7. Save the model

In [39]:
import pickle

pickle.dump(rf, open("model.pkl", "wb"))

In [40]:
with open("model.pkl", "wb") as f:  # context manager
    pickle.dump(rf, f)

In [41]:
ls -l *.pkl

 Datentr„ger in Laufwerk C: ist Windows
 Volumeseriennummer: 4696-704D

 Verzeichnis von C:\Users\team-\Desktop\ml_code


 Verzeichnis von C:\Users\team-\Desktop\ml_code

26.06.2025  15:24           836.280 model.pkl
               1 Datei(en),        836.280 Bytes
               0 Verzeichnis(se), 16.613.781.504 Bytes frei
